In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
n_points = 50
n_samples = 1
vec_alpha = []
for alpha_value in np.linspace(0, 1, n_points):
	for n in range(n_samples):
		vec_alpha.append(alpha_value)

vec_alpha = np.array(vec_alpha)

EL = -60*mV
EE = 0*mV
EI = -80*mV
tauA = 1*ms
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms
vt = -50*mV
nMg = 1*mM
gamma = 3.57*mM
b = 0.062/mV

eqs = '''
dv/dt = ((EL - v) + ((1 - alpha)*gA + alpha*s*gN)*(EE - v) + gI*(EI - v))/tauL : volt (unless refractory)
dgA/dt = -gA/tauA : 1
dgN/dt = -gN/tauN : 1
dgI/dt = -gI/tauI : 1
s = 1/(1+(nMg/gamma)*exp(-b*v)) : 1
alpha : 1
'''

rate_list = [5, 10, 20]
firing_rate_alpha = {'alpha':vec_alpha, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}
CVISI_alpha = {'alpha':vec_alpha, 5:np.zeros(n_points*n_samples), 10:np.zeros(n_points*n_samples), 20:np.zeros(n_points*n_samples)}

for rate in rate_list:
	print(rate, end='\t')
	for i, alpha_value in enumerate(vec_alpha):
		start_scope()
		print(i/size(vec_alpha), end=' ')
		defaultclock.dt = 0.01*ms
		G = NeuronGroup(1, eqs, threshold='v>vt', reset='v=EL', refractory=2*ms, method='rk4')
		GPA = PoissonGroup(400, rates=rate*Hz)
		GPN = PoissonGroup(400, rates=rate*Hz)
		GPI = PoissonGroup(100, rates=rate*Hz)

		SA = Synapses(GPA, G, 'w : 1', on_pre='gA += w')
		SN = Synapses(GPN, G, 'w : 1', on_pre='gN += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SA.connect()
		SN.connect()
		SI.connect()

		G.v = EL
		SA.w = 0.1
		SN.w = 0.1
		SI.w = 0.4

		G.alpha = alpha_value
			
		run(5000*ms)
		
		spkM = SpikeMonitor(G)

		run(10000*ms)
		
		train = spkM.spike_trains()
		ISI = np.diff(train[0])

		firing_rate_alpha[rate][i] = spkM.num_spikes/10.0
		CVISI_alpha[rate][i] = ISI.var()/ISI.mean()**2
    
	print('\n')

5	0.0 

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/brian2/units/fundamentalunits.py:252: RuntimeWarning: Degrees of freedom <= 0 for slice
  return Quantity(func(ar, *args, **kwds),
 [py.warnings]
  return Quantity(func(ar, *args, **kwds),

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
 [py.warnings]
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 [py.warnings]
  ret = ret.dtype.type(ret / rcount)

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/brian2/units/fundamentalunits.py:229: RuntimeWarning: Mean of empty slice.


0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 

10	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 

20	0.0 0.02 0.04 0.06 0.08 0.1 0.12 0.14 0.16 0.18 0.2 0.22 0.24 0.26 0.28 0.3 0.32 0.34 0.36 0.38 0.4 0.42 0.44 0.46 0.48 0.5 0.52 0.54 0.56 0.58 0.6 0.62 0.64 0.66 0.68 0.7 0.72 0.74 0.76 0.78 0.8 0.82 0.84 0.86 0.88 0.9 0.92 0.94 0.96 0.98 



In [5]:
fRate_data_path = "../../Data/fRate_data/"
V_data_path = "../../Data/V_data/"

# Convert to pandas DataFrame
df_firing_rate = pd.DataFrame.from_dict(firing_rate_alpha)
df_CVISI = pd.DataFrame.from_dict(CVISI_alpha)

# Save Dataframe do csv file
df_firing_rate.to_csv(f'{fRate_data_path}firing_rate_iRate_fullNMDA_rate.dat')
df_CVISI.to_csv(f'{fRate_data_path}CVISI_ta_iRate_fullNMDA_rate.dat')